In [ ]:
from parse_obsidian_vault import ObsidianVault, MarkdownThenRecursiveSplit
from pprint import pprint
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import InfinityEmbeddings, InfinityEmbeddingsLocal

import pandas as pd

In [ ]:
infinity_api_url = "http://127.0.0.1:7997"

embeddings = InfinityEmbeddings(
    model="BAAI/bge-small-en-v1.5", infinity_api_url=infinity_api_url
)

In [ ]:
vault_path = "/home/xoph/SlipBoxCopy/Slip Box"
# vault_path = "/home/xoph/repos/github/nfroseth/world_graph/test_vault"
percentile_semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
vault_percent = ObsidianVault(vault_path=vault_path, vault_name="TEST_VAULT")
vault_percent.parse_obsidian_vault(splitter = percentile_semantic_chunker)
parsed_notes_percent = vault_percent.parsed_notes

In [ ]:
vault_path = "/home/xoph/SlipBoxCopy/Slip Box"

# BREAKPOINT_DEFAULTS: Dict[BreakpointThresholdType, float] = {
#     "percentile": 95,
#     "standard_deviation": 3,
#     "interquartile": 1.5,
# }

semantic_chunk_args = {
    # "percentile":{"breakpoint_threshold_type":"percentile"},
    "standard_deviation":{"breakpoint_threshold_type":"standard_deviation"},
    "interquartile":{"breakpoint_threshold_type":"interquartile"},
    # "gradient":{"breakpoint_threshold_type":"gradient"},
}
perentages = [i*5 for i in range(0, 20)][::-1]
percentile_chunk_args = {
    f"percentile_{i}": {"breakpoint_threshold_type":"percentile", "breakpoint_threshold_amount":i} for i in perentages
}
semantic_chunk_args.update(percentile_chunk_args)

splitters = {name: SemanticChunker(embeddings, **kwargs) for name, kwargs in semantic_chunk_args.items()}
splitters["mdThenRecursive"] = MarkdownThenRecursiveSplit()

In [ ]:
print(list(splitters.keys()))

In [ ]:
chunk_results = {}
for name, splitter in splitters.items():
    vault = ObsidianVault(vault_path=vault_path, vault_name="TEST_VAULT")
    try:
        vault.parse_obsidian_vault(splitter = splitter)
        chunk_results[name] = vault.parsed_notes
    except Exception as e:
        print(f"Failed on {name} with {e}")

In [ ]:
chunk_lengths = pd.DataFrame({name:{k: len(v.chunks) for k,v in notes.items()} for name, notes in chunk_results.items()})

In [ ]:
chunk_lengths

In [ ]:
for col in chunk_lengths:
    print(chunk_lengths[col].describe())

In [ ]:
chunk_lengths.sum()

In [ ]:
chunk_lengths[["percentile_95", "percentile_65", "percentile_60", "percentile_55", "percentile_50", "percentile_0", "mdThenRecursive", "interquartile"]].describe()

In [ ]:
pd.DataFrame(chunk_lengths[["percentile_95", "percentile_65", "percentile_60", "percentile_55", "percentile_50", "percentile_0", "mdThenRecursive", "interquartile"]].sum()).T

In [ ]:
chunk_lengths["per_60 minus mdRec"] = chunk_lengths["percentile_60"] - chunk_lengths["mdThenRecursive"]

In [ ]:
chunk_difference.describe()

In [ ]:
chunk_lengths[["per_60 minus mdRec", "percentile_60", "mdThenRecursive"]].plot.hist(bins=50, alpha=0.5)

In [ ]:
chunk_difference.describe()

In [ ]:
sorted_diff = chunk_lengths["per_60 minus mdRec"].sort_values()

In [ ]:
sorted_diff

In [ ]:
most_ext_diff = pd.concat([sorted_diff[:10], sorted_diff[-10:]])

In [ ]:
most_ext_diff #chunk_lengths["percentile_60"] - chunk_lengths["mdThenRecursive"]

In [ ]:
chunk_results['percentile_60']